# **Examples from JSC article**

In this notebook we reproduce all the computations done throughout the original article (see [this link]() for the JSC version and [this link](https://arxiv.org/abs/2202.05550) for the arXiV version). This showcase how to use the package `pseries_basis` in practice.

In [1]:
# Loading the package
from pseries_basis import *
from ore_algebra import *
n = PSBasis.n(PSBasis)

### **Examples 5 and 7**

In this examples we explored the basic properties of the power basis $\mathcal{P} = \langle x^k\rangle_{k=0}^\infty$ and the binomial basis $\mathcal{C} = \langle\binom{x}{k}\rangle_{k=0}^\infty$.

These to basis are easy to get in our package by the classes `PowerBasis` and `BinomialBasis`:

In [2]:
P = PowerBasis()
P[:10]

[1, x, x^2, x^3, x^4, x^5, x^6, x^7, x^8, x^9]

In [3]:
C = BinomialBasis()
C[:4]

[1, x, 1/2*x^2 - 1/2*x, 1/6*x^3 - 1/2*x^2 + 1/3*x]

In example 5, we stated that both sequences are **factorial bases**. This means that the elements in the sequences can be obtained by multiplying the polynomial by a new polynomial of degree 1. 

**Factorial bases** has their own structure in the package (see [documentation](https://antonio-jp.github.io/pseries_basis/pseries_basis/factorial_basis.html#pseries_basis.factorial_basis.FactorialBasis)) and include plenty of methods related with this feature.

In [4]:
isinstance(P, FactorialBasis)

True

In [5]:
isinstance(C, FactorialBasis)

True

In Example 7 we showed how the root sequence and leading coefficient sequences defined the bases $\mathcal{P}$ and $\mathcal{C}$. These two properties can be obtained from the code using the methods ``root_sequence`` and ``leading_coefficient`` or the attributes ``rho`` and ``cn`` (following the notation from the paper):

Root sequence and leading coefficient sequence for $\mathcal{P}$ using the methods:

In [6]:
rho_P = P.root_sequence()
print([rho_P(i) for i in range(10)])
cn_P = P.leading_coefficient()
print([cn_P(i) for i in range(10)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


Root sequence and leading coefficient sequence for $\mathcal{C}$ using the attributes:

In [7]:
rho_C = C.rho
print([rho_C(i) for i in range(10)])
cn_C = C.cn
print([cn_C(i) for i in range(10)])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 1, 1/2, 1/6, 1/24, 1/120, 1/720, 1/5040, 1/40320, 1/362880]


### **Example 10**

Example 10 shows the basic compatibilities for the power ($\mathcal{P}$) and binomial ($\mathcal{C}$) bases of the basic operators $E: x \mapsto x+1$ and $D = \partial_x$.

In ``pseries_basis``, the compatibilities for default bases are automatically built, but the user can always set up more compatibilities on the go by providing the compatibility equation. 

The method `compatibility`, after receiving an operator $L$, returns the compatibility in the following format:
$$(A, B, m, \alpha),$$
where 
* $A,B$ are non-negative integers, are the bounds of the compatibility.
* $m$ is a positive integer, meaning the number of sections for compatibility.
* $\alpha(j,i,k) = \alpha_{k,j,i}$ is a three-indexed sequence, are the compatibility coefficients.
Such that, for all $k\in \mathbb{N}$ and $r \in \{0,\ldots,m-1\}$ the following identity holds (see definition 50 in the [paper](https://arxiv.org/abs/2202.05550)):
$$L P_{km+r}(x) = \sum_{i=-A}**B \alpha_{k,j,i} P_{km+r+i}(x).$$

**Compatibilities for $\mathcal{P}$**

In [8]:
P.compatible_operators()

dict_keys(['x', 'Id', 'Dx'])

Here we see how the power basis has compatibility with 3 different operators:
* Multiplication by $x$
* The identity operator $Id$
* The standard derivation $\partial_x$ or $D$.

Let us see the compatibility for the standard derivation:

In [9]:
A,B,m,alpha = P.compatibility('Dx')
print(f"{A=}; {B=}; {m=}")
print(f"alpha = {[alpha(0,i,n) for i in range(-A, B+1)]}")

A=1; B=0; m=1
alpha = [n, 0]


This means that the following identity holds:
$$\partial_x (x^k) = k(x^{k-1}),$$
which is the usual definition of a derivation. We can check this identity up to $k = 100$ using the following code:

In [10]:
all(sum(alpha(0, i, k)*P[k+i] for i in range(-A, B+1)) == P[k].derivative() for k in range(A,100))

True

**Compatibilities for $\mathcal{C}$**

In [11]:
C.compatible_operators()

dict_keys(['x', 'Et', 'E'])

In this case, we obtain three different operators:

* Multiplication by $x$.
* A special shift operator $E_t$ (in this case equivalent to $E$)
* The usual shift operator $E: x \mapsto x+1$.

We recommend seeing the [official documentation for the Binomial Basis](https://antonio-jp.github.io/pseries_basis/pseries_basis/factorial_basis.html#pseries_basis.factorial_basis.BinomialBasis) to see how this operator $E_t$ is usually considered.

Let us study now the compatibility of $\mathcal{C}$ with the natural shift $E: x \mapsto x+1$:

In [12]:
A,B,m,alpha = C.compatibility('E')
print(f"{A=}; {B=}; {m=}")
print(f"alpha = {[alpha(0,i,n) for i in range(-A, B+1)]}")

A=1; B=0; m=1
alpha = [1, 1]


which means that the following identity holds:
$$ E \binom{n}{k} = \binom{x+1}{k} = \binom{x}{k-1} + \binom{x}{k},$$
which is a [classical identity for the binomials](https://en.wikipedia.org/wiki/Binomial_coefficient#Recursive_formula). We can check this identity up to $k = 100$ using the following code:

In [13]:
x = C[1].parent().gens()[0] # getting the value for x for the substitution
all(sum(alpha(0, i, k)*C[k+i] for i in range(-A, B+1)) == C[k](x+1) for k in range(A,100))

True

### **Example 23**

In Example 23 we compute the equivalent recurrence operators related with different operators for the factorial basis $\mathcal{P}$ and $\mathcal{C}$. This recurrence operator can be obtained using the method `recurrence` from a basis. The output of method `recurrence` is an [Ore Operator](https://github.com/mkauers/ore_algebra) in the operators $S_n$ and $S_n^{-1}$ (represented in the code with ``Sn`` and ``Sni``) which are the natural and inverted shift w.r.t. $n$:

**Recurrences for $\mathcal{P}$**

In [14]:
P.recurrence('Dx') # recurrence for the derivation

(n + 1)*Sn

In [15]:
P.recurrence('x') # recurrence for the multiplication by 'x'

Sni

**Recurrences for $\mathcal{C}$**

In [16]:
C.recurrence('E') # recurrence for shift E: x --> x + 1

Sn + 1

In [17]:
C.recurrence('x') # recurrence for the multiplication by 'x'

n*Sni + n

### **Example 25**

Example 25 is the firs time we use the compatibility ideas to solve some problems or to prove some identities. For doing so, we first consider difference operators in $E$ and $x$ and compute its associated recurrence using the method `recurrence`. Then we analyze the solutions of the original recurrence, the obtained recurrence and draft a conclusion.

**Setting up the input operators**

The first step is to create the input operators. For doing so, we are going to use the [``ore_algebra``](https://github.com/mkauers/ore_algebra) package:

In [18]:
OE = OreAlgebra(QQ[x], ('E', lambda p : p(x=x+1), lambda p : 0))
E = OE.gens()[0]; x = OE.base().gens()[0]

##### **Example 25.1**

The input operator is $L = E - c$ for a fixed constant. We can see we get for different values of $c$ the associated operator $\tilde{L} = S_n - (c-1)$:

In [19]:
C.recurrence(E - 3)

Sn - 2

In [20]:
C.recurrence(E - 10)

Sn - 9

In [21]:
C.recurrence(E + 1)

Sn + 2

The solution to the recursive equation $L = E - c$ is simple: a formal power series $f(x)$ such that $f(n) = pc^n$ for a constant $p$. If we write this formal power series in the form:
$$f(x) = \sum_{k=0}^\infty a_k \binom{x}{k},$$
then the sequence $a_k$ satisfies the associate recurrence:
$$\tilde{L} a_k = a_{k+1} - (c-1)a_k.$$
As we have just mentioned, the solution to this recurrence is $a_k = p(c-1)^k$, so for a fixed $n \in \mathbb{N}$ we have shown:
$$f(n) = pc^n = \sum_{k=0}^\infty p(c-1)^k \binom{n}{k} = p\sum_{k=0}^n (c-1)^k \binom{n}{k},$$
or more concisely, for all $c \in \mathbb{K}$:
$$c^n = \sum_{k=0}^n (c-1)^k\binom{n}{k}.$$

##### **Example 25.2**

In this example we took the operator $L = E^2 - 2E +1$. We can compute easily its associated recurrence operator:

In [22]:
C.recurrence(E**2 - 2*E + 1)

Sn^2

The conclusion of this example is quite straightforward, since the solutions to the obtained recurrence are very limited:
$$a = (a_0, a_1, 0, 0, 0, \ldots).$$
Using then the equivalence, let $f(x)$ be the solution to $L f(x) = 0$ and let $f(x) = \sum_{k=0}^\infty a_k \binom{x}{n}$. Then, the function $f(x)$ has the form:
$$f(x) = a_0 + a_1 \binom{x}{1} = a_0 + a_1x$$

We can check these are always solution to $L f(x) = 0$:
$$\begin{array}{rl}
    L f(x) & = (a_0 + a_1(x+2)) - 2(a_0 + a_1(x+1)) + (a_0 + a_1x) \\
           & = (a_0 - 2 a_0 + a_0) + (a_1 - 2a_1 + a_1)x + (2a_1 - 2a_1) \\
           & = 0
\end{array}$$

##### **Example 25.3**

In this example we considered the operator $L = E^ 2 - E - 1$. Using the method `recurrence`, we can obtain again its associated recurrence operator:

In [23]:
C.recurrence(E**2 - E - 1)

Sn^2 + Sn - 1

In this case we know a solution for the equation $L f(x) = 0$: a formal power series $f(x) = \sum_{n\geq 0} F_n x^n$ where $F_n$ is a sequence of Fibonacci numbers.
On the other hand, if we solve the recurrence $\tilde{L}$, then we obtain solutions of the form:
$$a_k = (-1)^n (C_1F_n + C_2F_{n+1}),$$
where $C_1$ and $C_2$ are two constants and $F_n$ are again the Fibonacci numbers (i.e., the sequence $0,1,1,2,...$). Hence, putting both solutions together, we obtain that, for all $n \in \mathbb{N}$ the following identity holds:
$$F_n = \sum_{k=0}^n (-1)^{k+1}F_k\binom{n}{k},$$
which is a [well-known identity](https://fungrim.org/entry/f95561/) for the Fibonacci numbers.

We can easily prove another two identities of the Fibonacci numbers using this technique. Consider the subsequences $G_n = F_{2n}$ and $H_n = F_{3n}$. They satisfies the following recurrences:

$$G_{n+2} = 3G_{n+1} - G_n,$$
$$H_{n+2} = 3H_{n+1} + H_n.$$

Let $L_G$ and $L_H$ be the annihilators for those recurrences. Let us use the method ``recurrence`` to compute their associated recurrences:

In [24]:
L_G = E**2 - 3*E + 1
print(f"Ass. recurrence for G(n): {C.recurrence(L_G)}")
L_H = E**2 - 4*E - 1
print(f"Ass. recurrence for H(n): {C.recurrence(L_H)}")

Ass. recurrence for G(n): Sn^2 - Sn - 1
Ass. recurrence for H(n): Sn^2 - 2*Sn - 4


The recurrence $\tilde{L}_G$ is exactly the recurrence for the Fibonacci numbers! Moreover, if we study the recurrence $\tilde{L}_H$, we can see that the sequence $2^kF_k$ is a solution. Hence, we have shown two other known identities (see [here](https://fungrim.org/entry/ac4d13/) and [here](https://fungrim.org/entry/d454a3/)):
$$G_n = F_{2n} = \sum_{k=0}^n F_k \binom{n}{k},$$
$$H_n = F_{3n} = \sum_{k=0}^n 2^kF_k \binom{n}{k}.$$

##### **Example 25.4**

Now it is time for considering some recurrences $L$ that has some non-constant coefficients. In Example 25.4 we started with the operator of the factorial sequence:
$$L = E - (x+1)$$
The associated recurrence is the following:

In [25]:
C.recurrence(E - (x+1))

Sn + (-n)*Sni + (-n)

This recurrence implies that any solution $c_k$ to $\tilde{L}$ has $c_1 = 0$. Moreover, if we shift it backwards once we get:

In [26]:
C.reduce_SnSni(C.Sni()*C.recurrence(E - (x+1)))

(-n + 1)*Sni^2 + (-n + 1)*Sni + 1

Which means that the solutions $c_k$ satisfy:
$$c_k = (n-1)(c_{k-1} + c_{k-2}).$$
The general solution to this recurrence is:
$$c_k = k!\left(C_1 \sum_{l=0}\frac{(-1)^l}{l!} + C_2\right).$$

If we consider the value $c_1 = 0$, then we obtain $C_2 = 0$. Hence, we obtain:
$$n! = \sum_{k=0}^n k! \left(\sum_{l=0}^k\frac{(-1)^l}{l!}\right)\binom{n}{k}.$$

##### **Example 25.5**

This example is the first where the original recurrence equation has no Liouvillian solution. The input operator is:
$$L = E^3 - (x^2 + 6x +10)E^2 + (x+2)(2x+5)E - (x+1)(x+2).$$

Again, by using the method `recurrence` we can get the recurrence operator associated with this $L$:

In [27]:
C.recurrence(E**3 - (x**2 + 6*x +10)*E**2 + (x+2)*(2*x+5)*E - (x+1)*(x+2))

Sn^3 + (-n^2 - 6*n - 7)*Sn^2 + (-2*n^2 - 8*n - 7)*Sn + (-n^2 - 2*n - 1)

Using other algorithms, we can see that this recurrence has a hypergeometric solution like $c_k = (k!)^2$. Hence, we can describe a non-Liouvillian solution to $L f(x) = 0$, with the explicit formula:
$$f(n) = \sum_{k=0}^n (k!)^2\binom{n}{k}.$$

### **Example 37**

In Example 37, we study the first product basis by extending the kernel $\binom{x}{n}^2$ to a polynomial basis.
Product bases can be build in the system by using the class `ProductBasis`. This class will extend the compatibility of all given operators automatically from the original compatibility in the basic bases.

However, we need to mention the type of operator we want to extend dividing them into endomorphism and derivations. The method of extension depends on the type of operator (the endomorphism case was shown in Theorem 55 of the [paper](https://arxiv.org/abs/2202.05550)). 

To build the basis $C_{(1,1),(0,0)}$ the following line suffices:

In [28]:
C_11_00 = ProductBasis([C, C], ends={'E'})
C_11_00

ProductBasis
	- Binomial basis (x) choose n
	- Binomial basis (x) choose n

**Compatibility with $E$**

We can check the compatibility equation for the shift operator:

In [29]:
A,B,m,alpha = C_11_00.compatibility('E')
print(f"{A=}; {B=}; {m=}")
print(f"alpha = \n{[[alpha(j,i,n) for i in range(-A, B+1)] for j in range(m)]}")

A=2; B=0; m=2
alpha = 
[[1, 2, 1], [n/(n + 1), (2*n + 1)/(n + 1), 1]]


We can see now that the value of $m$ is 2, meaning that the compatibility is defined in two sections: for the elements $2k$ and for the elements $2k+1$.

Let us check equation (13) in the [paper](https://arxiv.org/abs/2202.05550). In this equation we are checking compatibility of the even terms $P_{2k}(x)$ (i.e, the first section with $j=0$):

In [30]:
[alpha(0,i,n) for i in range(-A, B+1)] 

[1, 2, 1]

This means precisely that equation (13) holds:
$$P_{2k}(x+1) = 1 P_{2k-2}(x) + 2 P_{2k-1}(x)  + 1 P_{2k}(x).$$

We can also check the coefficients from equation (15). Now we are looking for the compatibility for the odd terms $2k+1$, so we need to look to the section with $j=1$:

In [31]:
alpha(1,0,n), alpha(1,-1,n), alpha(1,-2,n)

(1, (2*n + 1)/(n + 1), n/(n + 1))

From these coefficients, we can write the recurrence the odd terms of the basis satisfy (which is exactly (15)):
$$P_{2k+1}(x+1) = 1 P_{2k-1}(x) + \frac{2k + 1}{k+1} P_{2k}(x) + \frac{k}{k+1}P_{2k+1}(x).$$

**Compatibility with $X$:**

In Example 37 we also shown the formulas for compatibility for th multiplication by $x$. We can see down here that both formulas from the [paper](https://arxiv.org/abs/2202.05550) can be recovered with the following piece of code:

In [32]:
A,B,m,alpha = C_11_00.compatibility('x')
print(f"{A=}; {B=}; {m=}")
print("alpha:")
print(f"first section -> {[alpha(0,i,n) for i in range(-A, B+1)]}")
print(f"second section -> {[alpha(1,i,n) for i in range(-A, B+1)]}")

A=0; B=1; m=2
alpha:
first section -> [n, n + 1]
second section -> [n, n + 1]


In this case we can see that the compatibility formulas in each section are the same:
$$xP_{2k}(x) = kP_{2k}(x)  + (k+1)P_{2k+1}(x),$$
$$xP_{2k+1}(x) = kP_{2k+1}(x)  + (k+1)P_{2k+2}(x).$$

## **Main examples of the paper**

This section includes the main examples from the [paper](https://arxiv.org/abs/2202.05550) (i.e., its section 6).

### **Example 44**

In this example we work with an operator $L$ of order $7$:
$$\begin{array}{rl}
L\ &=\ (x+8) (27034107689\, x+247037440535)\, E^7\\
&-2 (x+7) (27034107689\, x^2+707256640479\, x+3519513987204)\, E^6\\
&+(27034107689\, x^4+1763504948043\, x^3+29534526868562\, x^2\\
&\qquad\qquad\qquad +187161930754966\, x+404930820118700)\, E^5\\
&-4 (121973169216\, x^4+3928755304511\, x^3+43197821249228\, x^2\\
&\qquad\qquad\qquad +198945697078905\, x+329021406797184)\, E^4\\
&+(2167208392754\, x^4+45326791213914\, x^3+347739537911929\, x^2\\
&\qquad\qquad\qquad +1165212776491303\, x+1439937061155596)\, E^3\\
&-2 (613023852648\, x^4+8954947813901\, x^3+52565810509778\, x^2\\
&\qquad\qquad\qquad +141274453841469\, x+142893654078876)\, E^2\\
&-(x+2)^2 (1109455476579\, x^2+3624719391913\, x-357803625948)\, E\\
&+24 (x+1)^2 (x+2) (8996538731\, x+29816968829),
\end{array}$$

In [33]:
L = sum([
    E**7*(x+8)*(27034107689*x+247037440535),
    -E**6*2*(x+7)*(27034107689*x**2+707256640479*x+3519513987204),
    E**5*(27034107689*x**4+1763504948043*x**3+29534526868562*x**2+187161930754966*x+404930820118700),
    -E**4*4*(121973169216*x**4+3928755304511*x**3+43197821249228*x**2+198945697078905*x+329021406797184),
    E**3*(2167208392754*x**4+45326791213914*x**3+347739537911929*x**2+1165212776491303*x+1439937061155596),
    -E**2*2*(613023852648*x**4+8954947813901*x**3+52565810509778*x**2+141274453841469*x+142893654078876),
    -E*(x+2)*2*(1109455476579*x**2 + 3624719391913*x-357803625948),
    24*(x+1)*2*(x+2)*(8996538731*x+29816968829),
])

This operators is built using the multiplication by $x$ and the shift operator $E$ so it is certainly compatible with the binomial basis. We can try to see the compatibility recurrence with this basis:

In [34]:
C.remove_Sni(C.recurrence(L))

(27034107689*n^2 + 1058060671205*n + 10343839877166)*Sn^11 + (-54068215378*n^3 - 3171730280743*n^2 - 60367177203131*n - 375515714076170)*Sn^10 + (27034107689*n^4 + 2250118886445*n^3 + 63454052741427*n^2 + 750829363942371*n + 3216144625438506)*Sn^9 + (-244585707663*n^4 - 11109838955634*n^3 - 178441600717634*n^2 - 1167204758907073*n - 2452622856331934)*Sn^8 + (-762705145354*n^4 - 43913891082374*n^3 - 895087419553786*n^2 - 7794634090787260*n - 24735373253814280)*Sn^7 + (2554281137666*n^4 + 72370982033502*n^3 + 602132612364058*n^2 + 696251233492174*n - 6384359489665008)*Sn^6 + (14239397680488*n^4 + 469982105996824*n^3 + 5650012414735139*n^2 + 29337421015074157*n + 55568959092821460)*Sn^5 + (26715388355284*n^4 + 802568388226576*n^3 + 8795247582305557*n^2 + 41738197001213007*n + 72462258931901420)*Sn^4 + (26280214781962*n^4 + 676777333809862*n^3 + 6306708115151769*n^2 + 25231585803881087*n + 36605027752744940)*Sn^3 + (14432893593006*n^4 + 301499129167768*n^3 + 2248885677882753*n^2 + 71124883

In this case, it does not seem to help: we have obtained a recurrence of order 11 and the coefficients are not improving at all its size or complexity (in fact, it seems the degrees of the coefficients are simply increasing). In the paper, we tried the product basis $\mathcal{C}_{(1,1),(0,0)}$ that we have already built in this notebook.

Since it is a product basis, all the compatibilities are now described in 2 sections, and we obtain a matrix of recurrence for each of our basic operators:

In [35]:
## Equation 32
C_11_00.recurrence('E')

[                Sn + 1      (2*n + 1)/(n + 1)]
[                  2*Sn (n + 1)/(n + 2)*Sn + 1]

In [36]:
## Equation 33
C_11_00.recurrence('x')

[      n   n*Sni]
[(n + 1)       n]

We can then build a recurrence matrix for the operator $L$ by substituing in the operator $L$ the matrices we have just shown. This can be done in the code with the method `recurrence`:

In [37]:
M = C_11_00.recurrence(L)
L_00 = M[0,0]; L_01 = M[0,1]; L_10 = M[1,0]; L_11 = M[1,1]

## Writing the output in text files
def write_in_file(operator, file):
    with open(file, "w") as f:
        mons = operator.polynomial().monomials()
        coeffs = operator.coefficients()
        for i in range(len(mons)):
            f.write(f"({coeffs[i]})*{mons[i]} {'+' if i < len(mons)-1 else ''}\n")
    return

write_in_file(L_00, "example44/L_00.txt")
write_in_file(L_01, "example44/L_01.txt")
write_in_file(L_10, "example44/L_10.txt")
write_in_file(L_11, "example44/L_11.txt")

Now the variables $L_{ij}$ contains the operators described in equation (34). These operators are quite big, involving rational coefficients. We are going to print them here, but they are also available in the subfolder `example44` of the repository, each one in a `.txt` file with the corresponding name.

In [38]:
L_00

(27034107689*n^2 + 841787809693*n + 6544142915370)*Sn^7 + (-54068215378*n^9 - 657558265666*n^8 + 37095713911221*n^7 + 1025489135164547*n^6 + 10691142574920339*n^5 + 57323567313337619*n^4 + 169677116361660226*n^3 + 274018839251402700*n^2 + 220488594955202952*n + 67334168501422560)/(n^6 + 21*n^5 + 175*n^4 + 735*n^3 + 1624*n^2 + 1764*n + 720)*Sn^6 + (27034107689*n^9 - 2967463897532*n^8 - 177910103595067*n^7 - 3564893119708019*n^6 - 35529302717905762*n^5 - 196601860342462371*n^4 - 622714432849619000*n^3 - 1105255334070853214*n^2 - 1004763207156854068*n - 356970548465376240)/(n^5 + 15*n^4 + 85*n^3 + 225*n^2 + 274*n + 120)*Sn^5 + (1972211122835*n^8 + 96501192732394*n^7 + 1682092157093846*n^6 + 14210500613504874*n^5 + 66129117751682725*n^4 + 185086594141542412*n^3 + 326192458089200486*n^2 + 340475299799632436*n + 154217757549581808)/(n^4 + 10*n^3 + 35*n^2 + 50*n + 24)*Sn^4 + (-2972566483581*n^7 + 131524570383853*n^6 + 4516790093123463*n^5 + 46822106744903251*n^4 + 217975963180888146*n^3 + 476

In [39]:
L_01

(432545723024*n^3 + 13924454314467*n^2 + 132865603490137*n + 320663002853130)/(n + 7)*Sn^6 + (-811023230670*n^9 - 37537560577396*n^8 - 701377083344301*n^7 - 7013116159894573*n^6 - 42349048498079901*n^5 - 165298056655884679*n^4 - 430032837083767440*n^3 - 727181990061874632*n^2 - 709111257431946408*n - 288627075154225440)/(n^6 + 21*n^5 + 175*n^4 + 735*n^3 + 1624*n^2 + 1764*n + 720)*Sn^5 + (378477507646*n^9 + 9651817449260*n^8 - 84956276540988*n^7 - 4864102611835212*n^6 - 60612906473043864*n^5 - 359826098431415694*n^4 - 1142268348180551510*n^3 - 1935887247995744282*n^2 - 1598775392345052140*n - 486091649305683600)/(n^5 + 15*n^4 + 85*n^3 + 225*n^2 + 274*n + 120)*Sn^4 + (3985703076428*n^8 + 262554701903936*n^7 + 5600637162681716*n^6 + 57297215815442444*n^5 + 319287470855608696*n^4 + 1004783352889525384*n^3 + 1752288927398484324*n^2 + 1549175301973953672*n + 530815798417018032)/(n^4 + 10*n^3 + 35*n^2 + 50*n + 24)*Sn^3 + (-31542021389162*n^7 - 721177777654596*n^6 - 6018693139602036*n^5 - 2248

In [40]:
L_10

(432545723024*n^8 + 23224187359483*n^7 + 502488555653728*n^6 + 5710312952201326*n^5 + 37385228150050648*n^4 + 144628664112716707*n^3 + 322468734826398968*n^2 + 377024692155264852*n + 175280329796963040)/(n^6 + 27*n^5 + 295*n^4 + 1665*n^3 + 5104*n^2 + 8028*n + 5040)*Sn^7 + (-811023230670*n^8 - 38348583808066*n^7 - 721497569017449*n^6 - 7055433052082829*n^5 - 39388260638957573*n^4 - 128282198883459653*n^3 - 234900429440449300*n^2 - 213842222388469980*n - 68047095857571936)/(n^5 + 20*n^4 + 155*n^3 + 580*n^2 + 1044*n + 720)*Sn^6 + (378477507646*n^8 + 10219533710729*n^7 - 75115220337905*n^6 - 4953889797557169*n^5 - 62793464992380923*n^4 - 366656549667963608*n^3 - 1094530120869549178*n^2 - 1598526058285630416*n - 890711902164913736)/(n^4 + 14*n^3 + 71*n^2 + 154*n + 120)*Sn^5 + (3985703076428*n^7 + 268533256518578*n^6 + 5881896909629516*n^5 + 60941102732435774*n^4 + 333823647653768120*n^3 + 975936924004828404*n^2 + 1412686020604608428*n + 778778703650949592)/(n^3 + 9*n^2 + 26*n + 24)*Sn^4 + (

In [41]:
L_11

(27034107689*n^3 + 868821917382*n^2 + 7385930725063*n + 6544142915370)/(n + 8)*Sn^7 + (-54068215378*n^9 - 657558265666*n^8 + 52234814217061*n^7 + 1681006394079982*n^6 + 21792321065232130*n^5 + 152281425341639860*n^4 + 614728778531456855*n^3 + 1420138223211689168*n^2 + 1720566547214627076*n + 832541752971656640)/(n^6 + 27*n^5 + 295*n^4 + 1665*n^3 + 5104*n^2 + 8028*n + 5040)*Sn^6 + (27034107689*n^9 - 2967463897532*n^8 - 202511141592057*n^7 - 4645599827907501*n^6 - 54208096924525218*n^5 - 360843085732658375*n^4 - 1414637831335674934*n^3 - 3195312180012654656*n^2 - 3789790796103994360*n - 1790989640550298416)/(n^5 + 20*n^4 + 155*n^3 + 580*n^2 + 1044*n + 720)*Sn^5 + (1972211122835*n^8 + 106341607931190*n^7 + 2095782175045238*n^6 + 20209653012424702*n^5 + 104165543301301409*n^4 + 286259251251577612*n^3 + 373705459341213910*n^2 + 127075697605422184*n - 89841864856316632)/(n^4 + 14*n^3 + 71*n^2 + 154*n + 120)*Sn^4 + (-2972566483581*n^7 + 132432748660529*n^6 + 5503857165293235*n^5 + 69842247465

### **Example 45 and 49**

In Example 45, we consider the following operator $$L = (x+2)^2E^2 - (11x^2 + 33x+25)E - (x-1)^2.$$

Here we are interested in the kernel 
$$K(x,n) = \binom{x}{n}\binom{x+n}{2n}.$$

This kernel can not be built as a product basis of two other factorial basis. In Example 49 we described how we can build a basis to have this kernel as its elements: using a *Shuffled basis*. This kernel is composed of two main pieces:
$$\left\{\binom{x}{k}\right\}_k,\qquad \left\{\binom{x+k}{2k}, \binom{x+k}{2k+1}\right\}_k.$$

The first is the binomial basis (which we have already created in variable `C`). The second piece is more interesting by itself. However, it can be built from the code with the method `GeneralizedBinomial`. This method receives as arguments some values $a,b,c,m,r$ and builds a basis that has the binomial coefficients $$\binom{ax + bk + c}{mk+r},$$ as its elements. It also incorporates all the default compatibilities. See the [documentation](https://antonio-jp.github.io/pseries_basis/pseries_basis/misc.html#pseries_basis.misc.GeneralizedBinomial) for further information.

Then, the basis used for Example 45 is a $(2,1,2)$-shuffled basis of these two bases. We can produce this basis similarly to how we produced a `ProductBasis`: using the class `SievedBasis`:

In [42]:
B_45 = SievedBasis([C, GeneralizedBinomial(1,1,0,2,0)], [1,0,1], ends={'E'})

Using this basis we can check that every 3 terms we get an element of the kernel, i.e., if we denote the elements of the basis by $P_k(x)$, then:
    $$P_{3n}(x) = K(x,n) = \binom{x}{n}\binom{x+n}{2n}.$$

In [43]:
x = B_45[1].parent().gens()[0]
all(B_45[3*i] == binomial(x, i)*binomial(x+i,2*i) for i in range(10))

True

At this point, we can use the method recurrence to obtain a $3\times 3$ matrix of recurrences associated with the desired basis (by using the method `recurrence`):

In [44]:
L = (x+2)^2*E^2 - (11*x^2 + 33*x + 25)*E - (x+1)^2

In this problem we are only interested in expressing the solution in the following form:
$$y_n = \sum_{k=0}^n c_k K(n,k),$$
so, in terms of the recurrences obtained, we are only concerned with the recurrences obtaines in the first column. All those recurrence must be solved simultaneously by $(c_k)$, which means we can compute the `gcrd` and obtain a simpler recurrence for $(c_k)$:

In [45]:
M = B_45.recurrence(L)
first_column = [B_45.remove_Sni(M[j,0]) for j in range(M.nrows())]
gcrd = first_column[0].gcrd(*first_column[1:])
gcrd

(n + 1)*Sn - 4*n - 2

As shown in Example 45, this is the recurrence for the Catalan numbers, hence we obtain:
$$y_n = \sum_{k=0}^n \binom{2k}{k}\binom{n}{k}\binom{n+k}{2k} = \sum_{k=0}^n \binom{n}{k}^2 \binom{n+k}{k}.$$

This formula for $y_n$ is known as the Apéry's $\zeta(2)$-sequence. We have found using this method a closed-form formula to a solution of the original operator
$$L = (x+2)^2E^2 - (11x^2 + 33x+25)E - (x-1)^2.$$

### **Example 46**

Similarly to Example 45, in Example 46 we start with the following operator:
$$L = (x + 2)^3E^2 −(2x + 3)(17x^2 + 51x + 39)E + (x + 1)^3,$$
and we use the kernel:
$$K(x,n) = \binom{x+n}{2n}^2.$$

For this example, we can use a `ProductBasis` using as building blosk the basis created for $\binom{x+n}{2n}$ that we have seen before in Example 45 that can be created by using `GeneralizedBinomial`:

In [46]:
B2 = GeneralizedBinomial(1,1,0,2,0)
B_46 = ProductBasis([B2, B2], ends={'E'})

Since the generalized binomial has compatibilities in 2 sections, we get that the compatibility of the product basis has exactly 4 sections, meaning that using the method `recurrence` now will provide a $4\times 4$ matrix. Similarly to Example 45, we are only interested in solutions of the type
$$y_n = \sum_{k=0}^n c_k K(n,k) = \sum_{k=0}^n c_k \binom{n+k}{2n}^2,$$
so from the matrix obtained for the operator L, we are only interested in the `gcrd` of the first column. Then $(c_k)$ must satisfy this recurrence:

In [47]:
L = (x+2)**3*E**2 - (2*x+3)*(17*x**2+51*x+39)*E + (x+1)**3
M = B_46.recurrence(L)
first_column = [B_46.remove_Sni(M[j,0]) for j in range(M.nrows())]
gcrd = first_column[0].gcrd(*first_column[1:])
gcrd

(n^2 + 2*n + 1)*Sn - 16*n^2 - 16*n - 4

Using closure properties, we can check that a solution to this recurrence is the square of the catalan numbers $c_k = \binom{2k}{k}^2$. Hence, putting everything together, we obtain:
$$y_n = \sum_{k=0}^n \binom{2k}{k}^2 \binom{n+k}{2k}^2 = \sum_{k=0}^n \binom{n}{k}^2\binom{n+k}{k}^2.$$

This sequence is known as the Apéry's $\zeta(3)$-sequence, meaning we have found a close form for a solution of the operator 
$$L = (x + 2)^3E^2 −(2x + 3)(17x^2 + 51x + 39)E + (x + 1)^3.$$